In [88]:
import pandas as pd
%store -r operators_coverage

In [122]:
operators_coverage = operators_coverage.drop(columns=[col for col in columns_to_drop if col in operators_coverage.columns])

In [124]:
operators_coverage.columns = [
   'radio', 'mcc', 'net', 'area', 'cell',
    'unit', 'lon', 'lat', 'range', 'samples',
    'changeable', 'created', 'updated', 'average_signal'
]
display(operators_coverage.head())

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,average_signal
0,GSM,260,1,41007,32573,0,14.3456,53.0766,21667,138,1,1310099465,1714913476,0
1,GSM,260,1,41025,35441,0,14.3679,53.0655,17451,70,1,1310102445,1714913478,0
2,GSM,260,6,3235,39601,0,14.9897,51.1601,16958,50,1,1310566513,1727842141,0
3,GSM,260,6,1017,16444,0,19.3738,52.2280,6844,69,1,1311097775,1714504060,0
4,GSM,260,1,21025,40831,0,18.8648,50.2544,794,50,1,1311254920,1730666403,0


In [150]:
orange_gdf = orange_gdf.drop(columns=['geometry'])

In [152]:
from shapely.geometry import Point
import geopandas as gpd

In [158]:
orange_gdf['geometry'] = orange_gdf.apply(
    lambda row: Point(row['lon'], row['lat']),
    axis=1
)

In [159]:
orange_gdf = gpd.GeoDataFrame(operators_coverage, geometry='geometry')
orange_gdf.crs = "EPSG:4326"

In [162]:
orange_gdf = orange_gdf.to_crs(epsg=3857)

In [164]:
#1 meter ≈ 0.00001 degrees (latitude and longitude)
orange_gdf['geometry'] = orange_gdf.geometry.buffer(orange_gdf['range'])

In [200]:
orange_gdf = orange_gdf.to_crs(epsg=4326)

In [196]:
display(orange_gdf.head())

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,average_signal,geometry,operator
0,GSM,260,1,1.472481e+09,32573,0,14.3456,53.0766,21667,138,1,1310099465,1714913476,0,"POLYGON ((1618611.887 6997179.435, 1618507.555...",Orange
1,GSM,260,1,9.551963e+08,35441,0,14.3679,53.0655,17451,70,1,1310102445,1714913478,0,"POLYGON ((1616878.312 6995122.85, 1616794.28 6...",Orange
2,GSM,260,6,9.019891e+08,39601,0,14.9897,51.1601,16958,50,1,1310566513,1727842141,0,"POLYGON ((1685603.771 6649662.57, 1685522.114 ...",Play
3,GSM,260,6,1.469170e+08,16444,0,19.3738,52.2280,6844,69,1,1311097775,1714504060,0,"POLYGON ((2163525.551 6841456.233, 2163492.595...",Play
4,GSM,260,1,1.977393e+06,40831,0,18.8648,50.2544,794,50,1,1311254920,1730666403,0,"POLYGON ((2100813.93 6490450.564, 2100810.107 ...",Orange


In [192]:
operator_mapping = {
    1: 'Orange',
    2: 'T-Mobile',
    3: 'Plus',
    6: 'Play',
    7: 'Netia'
}
orange_gdf['operator'] = orange_gdf['net'].map(operator_mapping)

In [194]:
display(orange_gdf.head())

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,average_signal,geometry,operator
0,GSM,260,1,1.472481e+09,32573,0,14.3456,53.0766,21667,138,1,1310099465,1714913476,0,"POLYGON ((1618611.887 6997179.435, 1618507.555...",Orange
1,GSM,260,1,9.551963e+08,35441,0,14.3679,53.0655,17451,70,1,1310102445,1714913478,0,"POLYGON ((1616878.312 6995122.85, 1616794.28 6...",Orange
2,GSM,260,6,9.019891e+08,39601,0,14.9897,51.1601,16958,50,1,1310566513,1727842141,0,"POLYGON ((1685603.771 6649662.57, 1685522.114 ...",Play
3,GSM,260,6,1.469170e+08,16444,0,19.3738,52.2280,6844,69,1,1311097775,1714504060,0,"POLYGON ((2163525.551 6841456.233, 2163492.595...",Play
4,GSM,260,1,1.977393e+06,40831,0,18.8648,50.2544,794,50,1,1311254920,1730666403,0,"POLYGON ((2100813.93 6490450.564, 2100810.107 ...",Orange


In [176]:
print("Bounds of Orange GDF:", orange_gdf.total_bounds)


Bounds of Orange GDF: [13.96529088 48.78068835 23.98034806 54.85590276]


In [184]:
orange_gdf = orange_gdf.to_crs(epsg=3857)

#(area of the polygons)
orange_gdf['area'] = orange_gdf.geometry.area
display(orange_gdf[['range', 'area']].head())

CRS of Orange GDF: EPSG:4326


,range,area
0,21667,1.472481e+09
1,17451,9.551963e+08
2,16958,9.019891e+08
3,6844,1.469170e+08
4,794,1.977393e+06


In [202]:
orange_gdf.to_file('../outputs/operators_5g_coverage.geojson', driver='GeoJSON')